In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

!pip install -q pyspark==3.4.1
!pip install -q spark-nlp
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires pyspark[connect]~=3.5.1, but you have pyspark 3.4.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.4/744.4 kB 11.9 MB/s eta 0:00:00a 0:00:01


In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyngrok import ngrok

In [3]:
def main():
    
    # 1. Khởi tạo Spark Session với Spark NLP
    # Spark NLP sẽ tự động tải các JAR file cần thiết
    spark = sparknlp.start()
    print("Spark NLP Version:", sparknlp.version())
    print("Spark Version:", spark.version)
    !ngrok authtoken "35evQk9xMWrFeWUzTNPlnbYuLa7_66wFMqkNgoWcKz4YfUzDD"
    
    # Kill các tunnel cũ nếu bị treo
    ngrok.kill()
    
    # Mở tunnel tới cổng 4040 (Cổng mặc định của Spark UI)
    try:
        public_url = ngrok.connect(4040).public_url
        print(f"--------------------------------------------------")
        print(f"🚀 Spark UI đang chạy tại: {public_url}")
        print(f"👉 Bấm vào link trên để xem tiến độ TRƯỚC khi chạy Cell training bên dưới")
        print(f"--------------------------------------------------")
    except Exception as e:
        print("Không thể mở ngrok:", e)

    # 2. Tạo dữ liệu giả lập (Training Data)
    # Trong thực tế, bạn sẽ load từ HDFS, S3 hoặc file CSV lớn
    train_data = spark.read.csv("/kaggle/input/something/airline_sentiment.csv", header=True, inferSchema=True)
    train_data.select("text", "airline_sentiment").show(5)
    train_data.printSchema()
    train_data = train_data.dropna(subset=["text", "airline_sentiment"])
    train_data = train_data.withColumn("airline_sentiment", col("airline_sentiment").cast("string"))
    
    # 3. Xây dựng Pipeline xử lý NLP
    
    # Bước A: Chuyển text thành Document
    document_assembler = DocumentAssembler() \
        .setInputCol("text") \
        .setOutputCol("document")
    
    # Bước B: Tách từ (Tokenization)
    tokenizer = Tokenizer() \
        .setInputCols(["document"]) \
        .setOutputCol("token")
    
    # Bước C: Tải RoBERTa Base Embeddings (Pre-trained)
    # Model này sẽ download từ cache của Spark NLP
    bert_embeddings = RoBertaEmbeddings.pretrained("roberta_base", "en") \
        .setInputCols(["document", "token"]) \
        .setOutputCol("embeddings")
    
    # Bước D: Gom embeddings của token thành embeddings của cả câu (Pooling)
    sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")
    
    # Bước E: Fine-tune Classifier (ClassifierDL)
    # Đây là lớp mạng nơ-ron sẽ học cách phân loại dựa trên RoBERTa embeddings
    classifier = ClassifierDLApproach() \
        .setInputCols(["sentence_embeddings"]) \
        .setOutputCol("class") \
        .setLabelColumn("airline_sentiment") \
        .setMaxEpochs(10) \
        .setLr(0.001) \
        .setEnableOutputLogs(True)
    
        # Tạo Pipeline hoàn chỉnh
    nlp_pipeline = Pipeline(stages=[
        document_assembler,
        tokenizer,
        bert_embeddings,
        sentence_embeddings,
        classifier
    ])
    
    # 4. Train Model (Fit Pipeline)
    print("Bắt đầu training model...")
    model = nlp_pipeline.fit(train_data)
    print("Training hoàn tất! 🚀")
    
    save_path = "/kaggle/working/my_sentiment_model" 
    
    # Lệnh lưu model (overwrite để ghi đè nếu folder đã tồn tại)
    model.write().overwrite().save(save_path)
    
    print(f"Đã lưu model thành công tại: {save_path}")
        
    # 5. Kiểm thử (Inference)
    print("Demo test...")
    test_data = spark.createDataFrame([
        ("Welcom to Ha Noi.",),
        ("Weather in Da Nang is very chill!",),
        ("This is my 5th flight!",),
        ("Skibidi dom dom yet yet",),
        ("Vietnam Airline have very professional and dedicated service",),
        ("I always get delayed when flying Vietjet.",),
        ("My flight is terrible!",),
        ("Everything is OK!",)
    ]).toDF("text")
    
    predictions = model.transform(test_data)
    
    # Hiển thị kết quả
    predictions.select("text", "class.result").show(truncate=False)

if __name__ == "__main__":
    main()

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ca5c011d-bfeb-45f8-83cf-917f22eb9323;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.g

Spark NLP Version: 6.2.2
Spark Version: 3.4.1
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


--------------------------------------------------
🚀 Spark UI đang chạy tại: https://unlumped-vashti-criminologically.ngrok-free.dev
👉 Bấm vào link trên để xem tiến độ TRƯỚC khi chạy Cell training bên dưới
--------------------------------------------------
+--------------------+-----------------+
|                text|airline_sentiment|
+--------------------+-----------------+
|@VirginAmerica Wh...|          neutral|
|@VirginAmerica pl...|         positive|
|@VirginAmerica I ...|          neutral|
|"@VirginAmerica i...|         negative|
|@VirginAmerica an...|         negative|
+--------------------+-----------------+
only showing top 5 rows

root
 |-- id: string (nullable = true)
 |-- airline_sentiment: string (nullable = true)
 |-- negativereason: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- text: string (nullable = true)

roberta_base download started this may take some time.
Approximate size to download 284.2 MB
[ | ]roberta_base download started this may ta

25/11/19 03:49:30 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/11/19 03:49:30 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


[ | ]Download done! Loading the resource.
[ \ ]

[OK!]
Bắt đầu training model...


Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 64 - training_examples: 14640 - classes: 3
Epoch 1/10 - 2.13s - loss: 206.78072 - acc: 0.67354256 - batches: 229
Epoch 2/10 - 1.54s - loss: 182.81071 - acc: 0.7876462 - batches: 229
Epoch 3/10 - 1.59s - loss: 178.70853 - acc: 0.80973595 - batches: 229
Epoch 4/10 - 1.54s - loss: 176.7846 - acc: 0.81467015 - batches: 229
Epoch 5/10 - 1.53s - loss: 175.61772 - acc: 0.81953585 - batches: 229
Epoch 6/10 - 1.52s - loss: 174.82262 - acc: 0.8229624 - batches: 229
Epoch 7/10 - 1.56s - loss: 174.22006 - acc: 0.8261148 - batches: 229
Epoch 8/10 - 1.50s - loss: 173.72159 - acc: 0.82844484 - batches: 229
Epoch 9/10 - 1.51s - loss: 173.3036 - acc: 0.82940423 - batches: 229
Epoch 10/10 - 1.48s - loss: 172.95433 - acc: 0.8299525 - batches: 229
Training hoàn tất! 🚀
Đã lưu model thành công tại: /kaggle/working/my_sentiment_model
Demo test...


+------------------------------------------------------------+----------+
|text                                                        |result    |
+------------------------------------------------------------+----------+
|Welcom to Ha Noi.                                           |[neutral] |
|Weather in Da Nang is very chill!                           |[neutral] |
|This is my 5th flight!                                      |[neutral] |
|Skibidi dom dom yet yet                                     |[neutral] |
|Vietnam Airline have very professional and dedicated service|[positive]|
|I always get delayed when flying Vietjet.                   |[negative]|
|My flight is terrible!                                      |[negative]|
|Everything is OK!                                           |[positive]|
+------------------------------------------------------------+----------+

